In [4]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import Util
import linear_shrinkage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ret, ret_nonan, univ, tradeidx, dates = Util.load_data()

In [3]:
#Sanity check with Michael's code
today = tradeidx[0][0]
pastData = Util.get_past_period(h=0, T = 1260, N = 200, univ = univ, tradeidx = tradeidx, ret = ret)
investData = Util.get_invest_period(h=0, P = 1, N = 200, univ = univ, tradeidx = tradeidx, ret = ret)
combinedData = np.concatenate((pastData, investData))
print(combinedData.min())
print(combinedData.max())
print(combinedData.mean())

-0.31855
0.23618000000000003
0.0008989193208430914


In [13]:
def get_OOS_rets(cov_estimator, N, T, P, ret, univ, tradeidx, range_h = None):   
    if not range_h:
        range_h = len(univ)
    rets = []
    for h in range(range_h):
        pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
        cov = cov_estimator(pastRet)
        w = Util.optimal_weights(cov)
        outRet = Util.get_invest_period(h, P, N, univ, tradeidx, ret)
        curret = Util.OOS_rets(outRet, w)
        #curret = Util.retConstShare(outRet, w)
        rets.append(curret)
    return rets

In [14]:
N = 200
T = 1260
P = 1

In [20]:
equal_rets = get_OOS_rets(Util.identity_cov, 
                          N, T, P,
                          ret, univ, tradeidx,
                          range_h = None)

In [21]:
Util.print_normalize(equal_rets)

0.12918692971707574 0.18754260906922615


In [22]:
linear_shrinkage_rets = get_OOS_rets(linear_shrinkage.estimator, 
                                     N, T, P,
                                     ret, univ, tradeidx,
                                     range_h = None)

In [23]:
Util.print_normalize(linear_shrinkage_rets)

0.1111813838931928 0.12025630116520909
